In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.models import Model, Sequential

In [ ]:
df=pd.read_csv('train.csv')

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
cat=df.columns[1:11]
cont=df.columns[11:-1]

In [ ]:
cat

In [ ]:
cont

In [ ]:
df[cat[0]].unique()

In [ ]:
df.drop('id',axis=1,inplace=True)

In [ ]:
cor1=df.corr()

In [ ]:
import seaborn as sns
sns.heatmap(cor1)

In [ ]:
d1=dict()
for col in cat:
    d1[col]=df[col].unique()

In [ ]:
d1

In [ ]:
d2=dict()
for col in cont:
    d2[col]=(df[col].max(),df[col].min(),df[col].mean())
    

In [ ]:
d2

In [ ]:
df['cat9'].value_counts()

In [ ]:
df['cat5'].value_counts()

In [ ]:
df['cat7'].value_counts()

In [ ]:
len(df)

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
df_test=pd.read_csv('test.csv')

In [ ]:
for col in cat:
    lb=LabelEncoder()
    lb.fit(list(df[col].values)+list(df_test[col].values))
    df[col]=lb.transform(df[col].values)
    df_test[col]=lb.transform(df_test[col].values)

In [ ]:
df[cat]

In [ ]:

encoder = Sequential()
encoder.add(Dense(units=8,activation='relu',input_shape=(10,)))
encoder.add(Dense(units=6,activation='relu'))
encoder.add(BatchNormalization())
encoder.add(Dense(units=4,activation='relu'))

In [ ]:
decoder=Sequential()
decoder.add(Dense(units=6,activation='relu',input_shape=(4,)))
decoder.add(Dense(units=8,activation='relu'))
decoder.add(BatchNormalization())
decoder.add(Dense(units=10,activation='relu'))


In [ ]:
autoencoder=Sequential([encoder,decoder])
autoencoder.compile(loss='mse',optimizer='adam')


In [ ]:
A=np.array(df[cat])

In [ ]:
len(A)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
early_stop = EarlyStopping(verbose=1,monitor='loss', patience=20)

In [ ]:
autoencoder.fit(A,A,epochs=200, callbacks=[early_stop])

In [ ]:
autoencoder.summary()

In [ ]:
encoder.summary()

In [ ]:
Rep=encoder.predict(A)

In [ ]:
len(Rep)

In [ ]:
Rep.shape

In [ ]:
Rep_test=encoder.predict(np.array(df_test[cat]))

In [ ]:
X=pd.concat([pd.DataFrame(Rep),df[cont]],axis=1)

In [ ]:
X_test=pd.concat([pd.DataFrame(Rep_test),df_test[cont]],axis=1)

In [ ]:
X.head()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
y=df['target'].values

In [ ]:
y.min()

In [ ]:
febmodel = Sequential()
febmodel.add(Dense(units=10,activation='relu'))

febmodel.add(Dense(units=6,activation='relu'))

febmodel.add(Dense(units=4,activation='relu'))
febmodel.add(BatchNormalization())


febmodel.add(Dense(units=1,activation='relu'))
febmodel.compile(loss='mse', optimizer='adam')

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.25)

In [ ]:
s = EarlyStopping(verbose=1,monitor='val_loss', patience=50)

In [ ]:
X_train.isnull().sum()

In [ ]:
X_val.isnull().sum()

In [ ]:
pd.Series(y_train).isnull().sum()

In [ ]:
pd.Series(y_val).isnull().sum()

In [ ]:
X_train=np.array(X_train)
X_val=np.array(X_val)
y_train=np.array(y_train)
y_val=np.array(y_val)


In [ ]:
febmodel.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=300,callbacks=[s])

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
febmodel.save('model_feb21.h5')

In [ ]:
autoencoder.save('autoencoder_feb21.h5')

In [ ]:
newnn=load_model('model_feb21.h5')

In [ ]:
febmodel.save_weights('Downloads/tabular-playground-series-feb-2021')

In [ ]:
newauto=load_model('autoencoder_feb21.h5')

In [ ]:
febmodel.load_weights('Downloads/tabular-playground-series-feb-2021')

In [ ]:
febmodel.summary()

In [ ]:
X_test.isnull().sum()

In [ ]:
X_test=np.array(X_test)

In [ ]:
predictions=febmodel.predict(X_test)

In [ ]:
len(X_test)

In [ ]:
len(predictions)

In [ ]:
df_test.head()

In [ ]:
solution=pd.DataFrame(df_test['id'])

In [ ]:
len(predictions)

In [ ]:
predictions[0][0]

In [ ]:
z=np.array([predictions[i][0] for i in range(200000)])

In [ ]:
pd.Series(z)

In [ ]:
len(z)

In [ ]:
solution['target']=pd.Series(z)

In [ ]:
solution.to_csv('feb_solution21.csv',index=False)

In [ ]:
solution.head(-5)